In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("rawdata.xlsx")
df.head()

,date,time,sensor,location,number,activity,position,location.1
0,2024-01-16,10:10:30,0.0,A1,1,placed,inside,A1
1,2024-01-16,10:12:30,1.0,A2,1,picked,inside,A2
2,2024-01-16,10:28:30,1.0,A3,1,picked,Inside,A3
3,2024-01-16,10:38:20,0.0,A4,1,placed,Inside,A4
4,2024-01-16,10:45:20,1.0,A5,1,placed,inside,A5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        175 non-null    datetime64[ns]
 1   time        175 non-null    object        
 2   sensor      174 non-null    float64       
 3   location    175 non-null    object        
 4   number      175 non-null    int64         
 5   activity    175 non-null    object        
 6   position    175 non-null    object        
 7   location.1  175 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 11.1+ KB


In [4]:
df = df.drop(columns = ['sensor', 'location', 'number', 'location.1'], axis=1)

In [5]:
df['time'] = pd.to_timedelta(df['time'].astype(str))

In [6]:
df['DateTime'] = df['date'] + df['time']

In [7]:
df.head(10)

,date,time,activity,position,DateTime
0,2024-01-16,0 days 10:10:30,placed,inside,2024-01-16 10:10:30
1,2024-01-16,0 days 10:12:30,picked,inside,2024-01-16 10:12:30
2,2024-01-16,0 days 10:28:30,picked,Inside,2024-01-16 10:28:30
3,2024-01-16,0 days 10:38:20,placed,Inside,2024-01-16 10:38:20
4,2024-01-16,0 days 10:45:20,placed,inside,2024-01-16 10:45:20
5,2024-01-16,0 days 10:52:20,picked,inside,2024-01-16 10:52:20
6,2024-01-16,0 days 11:05:20,picked,inside,2024-01-16 11:05:20
7,2024-01-16,0 days 11:16:20,placed,inside,2024-01-16 11:16:20
8,2024-01-16,0 days 11:23:20,placed,inside,2024-01-16 11:23:20
9,2024-01-16,0 days 11:26:20,picked,inside,2024-01-16 11:26:20


In [8]:
df['Duration'] = df.groupby(['date', 'position'])['DateTime'].diff().fillna(pd.Timedelta(seconds=0))
df = df[df['Duration'] >= pd.Timedelta(seconds=0)]

In [9]:
df.head()

,date,time,activity,position,DateTime,Duration
0,2024-01-16,0 days 10:10:30,placed,inside,2024-01-16 10:10:30,0 days 00:00:00
1,2024-01-16,0 days 10:12:30,picked,inside,2024-01-16 10:12:30,0 days 00:02:00
2,2024-01-16,0 days 10:28:30,picked,Inside,2024-01-16 10:28:30,0 days 00:00:00
3,2024-01-16,0 days 10:38:20,placed,Inside,2024-01-16 10:38:20,0 days 00:09:50
4,2024-01-16,0 days 10:45:20,placed,inside,2024-01-16 10:45:20,0 days 00:32:50


In [10]:
total = df.groupby(['date', 'position'])['Duration'].sum().unstack().fillna(pd.Timedelta(seconds = 0))

In [11]:
activity_count = df.groupby(['date', 'activity'])['activity'].count().unstack().fillna(0)

In [12]:
total

position,Inside,inside,outside
date,,,
2024-01-16,0 days 00:09:50,0 days 15:24:50,0 days 00:00:00
2024-01-17,0 days 00:00:00,0 days 04:55:50,0 days 00:00:00
2024-01-18,0 days 10:46:30,0 days 11:53:00,0 days 04:08:40


In [13]:
activity_count

activity,picked,placed
date,,
2024-01-16,39,38
2024-01-17,10,7
2024-01-18,37,36
